In [1]:
import os
HOME = os.getcwd()
print("HOME:", HOME)

HOME: /Users/tarasbohdan/Desktop/projects/idea-contest-2023/ideation-api


In [2]:
!pip install -q 'git+https://github.com/facebookresearch/segment-anything.git'

You should consider upgrading via the '/Users/tarasbohdan/Desktop/projects/idea-contest-2023/ideation-api/venv/bin/python3.8 -m pip install --upgrade pip' command.


In [3]:
!mkdir -p {HOME}/weights
!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth -P {HOME}/weights

In [4]:
import os

CHECKPOINT_PATH = os.path.join(HOME, "weights", "sam_vit_h_4b8939.pth")
print(CHECKPOINT_PATH, "; exist:", os.path.isfile(CHECKPOINT_PATH))

/Users/tarasbohdan/Desktop/projects/idea-contest-2023/ideation-api/weights/sam_vit_h_4b8939.pth ; exist: True


In [5]:
import torch

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MODEL_TYPE = "vit_h"

/Users/tarasbohdan/Desktop/projects/idea-contest-2023/ideation-api/venv/lib/python3.8/site-packages/torch/nn/modules/transformer.py:20: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  device: torch.device = torch.device(torch._C._get_default_device()),  # torch.device('cpu'),


In [8]:
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH).to(device=DEVICE)

In [31]:
mask_generator = SamAutomaticMaskGenerator(sam)

In [15]:
!wget -q https://media.roboflow.com/notebooks/examples/dog.jpeg -P {HOME}/data
!wget -q https://media.roboflow.com/notebooks/examples/dog-2.jpeg -P {HOME}/data
!wget -q https://media.roboflow.com/notebooks/examples/dog-3.jpeg -P {HOME}/data
!wget -q https://media.roboflow.com/notebooks/examples/dog-4.jpeg -P {HOME}/data

In [32]:
import os

IMAGE_NAME = "dog.jpeg"
IMAGE_PATH = os.path.join(HOME, "data", IMAGE_NAME)

In [39]:
import cv2
import supervision as sv

image_bgr = cv2.imread(IMAGE_PATH)
image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

In [27]:
test = image_bgr.astype('float32')

In [29]:
print(image_rgb.shape)
print(image_rgb.dtype)

(1280, 720, 3)
uint8


In [36]:
import numpy as np

In [41]:
sam_result = mask_generator.generate(image_rgb)

RuntimeError: Could not infer dtype of numpy.uint8

In [ ]:
print(sam_result[0].keys())

In [ ]:
mask_annotator = sv.MaskAnnotator(color_lookup=sv.ColorLookup.INDEX)

detections = sv.Detections.from_sam(sam_result=sam_result)

annotated_image = mask_annotator.annotate(scene=image_bgr.copy(), detections=detections)

sv.plot_images_grid(
    images=[image_bgr, annotated_image],
    grid_size=(1, 2),
    titles=['source image', 'segmented image']
)